### Summary from part 1

In [ ]:
import random
random.randint(10,15)

In [ ]:
import pandas as pd 
import time as t
from lxml import html 
import requests
import random

user_agent = 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_6) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/12.1.2 Safari/605.1.15'
headers = {'User-Agent': user_agent}
urls = []
searchlink = 'https://www.yelp.com/search?find_desc=&find_loc=Clemson%2C+SC&ns=1'


# get page counts
page = requests.get(searchlink, headers = headers)
parser = html.fromstring(page.content)
page_nums = '//span[@class="lemon--span__373c0__3997G text__373c0__2pB8f text-color--normal__373c0__K_MKN text-align--left__373c0__2pnx_"]'
pg = parser.xpath(page_nums)

page_count = int(pg[0].text.split()[3])
suffix = ""
for i in range(1,(page_count + 1)):
    searchlink = 'https://www.yelp.com/search?find_desc=&find_loc=Clemson%2C+SC&ns=1' + suffix
    print(searchlink)
    page = requests.get(searchlink, headers = headers)
    parser = html.fromstring(page.content)
    businesslink=parser.xpath('//a[@class="lemon--a__373c0__IEZFH link__373c0__29943 link-color--blue-dark__373c0__1mhJo link-size--inherit__373c0__2JXk5"]')
    links = [l.get('href') for l in businesslink]
    true_links = [link for link in links if link.startswith('/biz') and not "hrid" in link]
    for link in true_links:
        url = 'https://www.yelp.com' + str(link)
        urls.append(url)
    t.sleep(random.randint(8,15))
    suffix = '&start=' + str(10 * i)

Check that the number of URLs generated are the same as the number of restaurants

In [ ]:
print(len(urls))

In [ ]:
for item in urls:
    print(item)

In the notebook for part 1, we still have not worked on textual analysis yet. We have not even mined the text yet. What we have done was to mine the URLs pointing to individual restaurants. These URLs are stored in a list, which is represented by variable **urls**. 

In this notebook, first we will download the reviews from the individual restaurants.

#### Patterns, patterns, and patterns

The principles are the same for the reviews. We want to go to the restaurant's page, identify the HTML tag/class containing the reviews, extract the components of the reviews (text, rating, date, ...), and finally, repeat the entire process across all the review pages. 

At the begining, we go through the same process at Part 1 to set up, with things like user-agent information and URL to restaurant's page. In this scenario, we will use the first item in the restaurant URL list. 

#### Load libraries

In [ ]:
import pandas as pd 
import time as t
from lxml import html 
from lxml import etree
import requests

#### Set initial variables

In [ ]:
user_agent = 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_6) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/12.1.2 Safari/605.1.15'
headers = {'User-Agent': user_agent}
searchlink = 'https://www.yelp.com/biz/yolk-asian-kitchen-clemson-3'

#### Scape the page

In [ ]:
page = requests.get(searchlink, headers = headers)
parser = html.fromstring(page.content)

#### Get the reviews

We want to get everything. In this case, it means that extracting a large blob of HTML codes, then go through the smaller tags individually. 

In [ ]:
reviews=parser.xpath('//div[@class="review review--with-sidebar"]')

In [ ]:
print(len(reviews))

In [ ]:
for review in reviews:
    temp = review.xpath('.//div[contains(@class, "biz-rating__stars")]/div')
    rating = [td.get('title') for td in temp]
       
    xpath_author  = './/a[@id="dropdown_user-name"]//text()'
    xpath_body    = './/p[@lang="en"]//text()'
    
    author  = review.xpath(xpath_author)
    date    = review.xpath('.//span[@class="rating-qualifier"]//text()')
    body    = review.xpath(xpath_body)
        
    # we are extracting the business title from the main page. 
    heading= parser.xpath('//h1[contains(@class,"biz-page-title embossed-text-white")]')
    bzheading = [td.text for td in heading]
    print (author, rating, date, body, bzheading)

Recalling the dataframe created at the begining of the part-1 notebook, and we never used it. We are using it now!

In [ ]:
reviews_df = pd.DataFrame()

for review in reviews:
    temp = review.xpath('.//div[contains(@class, "biz-rating__stars")]/div')
    rating = [td.get('title') for td in temp]
       
    xpath_author  = './/a[@id="dropdown_user-name"]//text()'
    xpath_body    = './/p[@lang="en"]//text()'
    
    author  = review.xpath(xpath_author)
    date    = review.xpath('.//span[@class="rating-qualifier"]//text()')
    body    = review.xpath(xpath_body)
        
    # we are extracting the business title from the main page. 
    heading= parser.xpath('//h1[contains(@class,"biz-page-title embossed-text-white")]')
    bzheading = [td.text for td in heading]
    
    # instead of printing out the values, we now create a dictionary, and append it 
    # to the data frame
    review_dict = {'restaurant' : bzheading,
                'rating': rating, 
                'author': author, 
                'date': date,
                'Review': body,
              }
    reviews_df = reviews_df.append(review_dict, ignore_index=True)

Check out the populated dataframe!

In [ ]:
reviews_df

Similar to the previous notebook, now that we get all the reviews on one page, we need to get the reviews on all pages. 

Find the patterns!

https://www.yelp.com/biz/yolk-asian-kitchen-clemson-3

https://www.yelp.com/biz/yolk-asian-kitchen-clemson-3?start=20

https://www.yelp.com/biz/yolk-asian-kitchen-clemson-3?start=40

...

In [ ]:
# get page counts
page = requests.get(searchlink, headers = headers)
parser = html.fromstring(page.content)
page_nums = '//div[@class="page-of-pages arrange_unit arrange_unit--fill"]'
pg = parser.xpath(page_nums)

In [ ]:
print(pg[0].text.split())

In [ ]:
page_count = int(pg[0].text.split()[3])
suffix = ""

# setup empty data frame for reviews
reviews_df = pd.DataFrame()
# loop through all review pages
for i in range(1,(page_count + 1)):
    searchlink = 'https://www.yelp.com/biz/yolk-asian-kitchen-clemson-3' + suffix
    print(searchlink)
    page = requests.get(searchlink, headers = headers)
    parser = html.fromstring(page.content)
    reviews=parser.xpath('//div[@class="review review--with-sidebar"]')    

    for review in reviews:
        temp = review.xpath('.//div[contains(@class, "biz-rating__stars")]/div')
        rating = [td.get('title') for td in temp]
       
        xpath_author  = './/a[@id="dropdown_user-name"]//text()'
        xpath_body    = './/p[@lang="en"]//text()'
    
        author  = review.xpath(xpath_author)
        date    = review.xpath('.//span[@class="rating-qualifier"]//text()')
        body    = review.xpath(xpath_body)
        
        # we are extracting the business title from the main page. 
        heading= parser.xpath('//h1[contains(@class,"biz-page-title embossed-text-white")]')
        bzheading = [td.text for td in heading]
    
        # instead of printing out the values, we now create a dictionary, and append it 
        # to the data frame
        review_dict = {'restaurant' : bzheading,
                    'rating': rating, 
                    'author': author, 
                    'date': date,
                    'Review': body,
                    }
        reviews_df = reviews_df.append(review_dict, ignore_index=True)
    t.sleep(5)
    suffix = '?start=' + str(20 * i)

In [ ]:
reviews_df.describe()

Now do it for ALL RESTAURANTS!

In [ ]:
for url in urls:
    print(url)

We have an additional outer loop to go through all the restaurants ...

In [ ]:
# we move the initial empty data frame outside, because
# we want all reviews from all restaurants to be stored in the same data frame. 
reviews_df = pd.DataFrame()

# loop through all restaurants
for url in urls:
    page = requests.get(url, headers = headers)
    parser = html.fromstring(page.content)
    page_nums = '//div[@class="page-of-pages arrange_unit arrange_unit--fill"]'
    pg = parser.xpath(page_nums)
    page_count = int(pg[0].text.split()[3])
    suffix = ""
    print("Restaurant",url,"has",page_count,"review pages")

    # loop through all review pages
    for i in range(1,(page_count + 1)):
        searchlink = url + suffix
        page = requests.get(searchlink, headers = headers)
        parser = html.fromstring(page.content)
        reviews=parser.xpath('//div[@class="review review--with-sidebar"]')    

        for review in reviews:
            temp = review.xpath('.//div[contains(@class, "biz-rating__stars")]/div')
            rating = [td.get('title') for td in temp]
       
            xpath_author  = './/a[@id="dropdown_user-name"]//text()'
            xpath_body    = './/p[@lang="en"]//text()'
    
            author  = review.xpath(xpath_author)
            date    = review.xpath('.//span[@class="rating-qualifier"]//text()')
            body    = review.xpath(xpath_body)
        
            # we are extracting the business title from the main page. 
            heading= parser.xpath('//h1[contains(@class,"biz-page-title embossed-text-white")]')
            bzheading = [td.text for td in heading]
    
            # instead of printing out the values, we now create a dictionary, and append it 
            # to the data frame
            review_dict = {'restaurant' : bzheading,
                        'rating': rating, 
                        'author': author, 
                        'date': date,
                        'Review': body,
                        }
            reviews_df = reviews_df.append(review_dict, ignore_index=True)
        t.sleep(random.randint(10,15))
        suffix = '?start=' + str(20 * i)

In [ ]:
reviews_df.to_csv("yelp_reviews_clemson_sc.csv", index = False)

In [ ]:
url

In [ ]:
reviews_df.describe()

Let's do some cleaning on our data ...

#### Remove invalid rows

In [ ]:
reviews_df.dropna()

#### Remove unnecessary symbols and phrases

In [ ]:
clean_reviews_df = pd.DataFrame()

In [ ]:
clean_reviews_df['Review'] = reviews_df.Review.str[2:-2]
clean_reviews_df['author'] = reviews_df.author.str[2:-2]
clean_reviews_df['date'] = reviews_df.date.str[12:-8]
clean_reviews_df['rating'] = reviews_df.rating.str[2:-2]
clean_reviews_df['restaurant'] = reviews_df.restaurant.str[16:-12]
clean_reviews_df['rating'] = reviews_df.rating.str[:1]

In [ ]:
reviews_df.head()

In [ ]:
reviews_df.to_csv("yelp_reviews_clemson_sc.csv", index = False)

#### Challenge
Perform the extraction of all review data from all restaurants in your home town and save it to a csv file. 

Provide answers to the following questions based on your collected data:

- Which restaurant has the highest average rating?
- Which restaurant has the lowest average rating?
- What is the average rating of your favorite restaurant?